# Testing the Triangloids in SPICE-3 Angled Probe Tip Sim
This notebook was used to plot the output of spice-3 triangloids in teh simulation window of the angled-tip input file. 

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import re
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.inputparser as inp
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

In [2]:
# test_dir = pth.Path('/home/jleland/data/external/spice/test/angled_test/')
test_dir = pth.Path('/home/jleland/data/external/spice/test/recessed_test/')
os.chdir(test_dir)

In [32]:
test_run_names = list(test_dir.glob('*'))
for i, trn in enumerate(test_run_names):
    print(i, trn)

0 /home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised
1 /home/jleland/data/external/spice/test/recessed_test/01_recessed


In [33]:
lowdens_dir = test_run_names[0]
lowdens_dir

PosixPath('/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised')

In [34]:
non_standard_variables = {'objects', 'objectsenum', 'edges', 'Pot'}
desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

In [35]:
for item in lowdens_dir.glob('*'):
    print(item)

/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/t-alpha_yz_-25.0.mat
/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/melange_1.slurm
/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/melange_0.slurm
/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/log.out
/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/log.err
/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/jobs.txt
/home/jleland/data/external/spice/test/recessed_test/02_recessed_centralised/input.inp


In [36]:
datafile_oi = 't-alpha_yz_-25.0.mat'
# tTfile_oi = 't-alpha_yz_-89.0T00.mat'

# tdata = sio.loadmat(datafile_oi, variable_names=non_standard_variables)
tdata = sio.loadmat(lowdens_dir / datafile_oi)
# tTdata = sio.loadmat(lowdens_dir / tTfile_oi)

In [37]:
print(tdata.keys())
# print(tTdata.keys())
# print(tTdata['alphaxz'])
# print(tTdata['alphayz'])

dict_keys(['__header__', '__version__', '__globals__', 'version', 'Nx', 'Ny', 'Nz', 'nospecies', 'procmax', 'historyntimes', 'count', 'Na', 'Np', 'Nc', 'Pot', 'Potvac', 'Ex', 'Ey', 'Ez', 'objects', 'edges', 'edgecharge', 'bcmatrix', 'snumber', 'objectscurrent', 'rho01', 'vx01', 'vy01', 'vz01', 'rho02', 'vx02', 'vy02', 'vz02'])


In [38]:
print(tdata['edges'].shape)

objectsenum = (tdata['objects'] > 0).astype(np.int64)
print(objectsenum.shape)

(257, 513, 129)
(257, 513, 129)


In [39]:
fig, ax = plt.subplots()
hand = ax.pcolormesh(objectsenum.sum(axis=2))
ax.axis('scaled')
ax.autoscale()
fig.tight_layout()
fig.colorbar(hand)

In [40]:
fig, ax = plt.subplots(figsize=[8,4])

ax.pcolormesh(np.transpose(objectsenum[128,:,:]))
ax.axis('scaled')
ax.autoscale()
fig.tight_layout()

In [41]:
fig, ax = plt.subplots(2, figsize=[10,10], sharex=True, sharey=True)

ax[0].pcolormesh(np.transpose(objectsenum[128,:,:]))
ax[1].pcolormesh(np.transpose(tdata['Pot'][128,:,:]))
for i in range(2):
    z_ext = 0.5
#     ax[i].quiver(8, 120, 1, 1 * np.tan(tTdata['alphayz']), scale=0.000000000000000005)

    ax[i].axis('scaled')
    ax[i].autoscale()
fig.tight_layout()


In [42]:
fig, ax = plt.subplots()

print(tdata['objectscurrent'][0][0].shape)
ax.plot(tdata['objectscurrent'][2][1])

(1,)


In [43]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')

print(objectsenum.shape)

X = np.arange(objectsenum.shape[0])
Y = np.arange(objectsenum.shape[1])
X, Y = np.meshgrid(Y, X)
print(X.shape)
print(Y.shape)
print(objectsenum.sum(axis=2).shape)

ax.plot_surface(X, Y, objectsenum.sum(axis=2))



(257, 513, 129)
(257, 513)
(257, 513)
(257, 513)


In [44]:
fig = plt.figure()
ax = fig.gca(projection='3d')

xp = []
yp = []
zp = []

for i in range(tdata['edges'].shape[0]):
    for j in range(tdata['edges'].shape[1]):
        for k in range(tdata['edges'].shape[2]):
            if tdata['edges'][i,j,k]:
                xp.append(i)
                yp.append(j)
                zp.append(k)

ax.scatter(xp[::10], yp[::10], zp[::10])

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/backends/_backend_tk.py", line 338, in idle_draw
    self.draw()
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/backends/backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/backends/backend_agg.py", line 388, in draw
    self.figure.draw(self.renderer)
  File "/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/matplotlib/artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  Fil

In [26]:
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.voxels(tdata['edges'])

KeyboardInterrupt: 